# Касьяненко Вера

# Работа с ресурсами

## Порядок сдачи домашнего

Под каждое домашнее вы создаете отдельную ветку куда вносите все изменения в рамках домашнего. Как только домашнее готово - создаете пулл реквест (обратите внимание что в пулл реквесте должны быть отражены все изменения в рамках домашнего). Ревьювера назначаете из таблицы - https://docs.google.com/spreadsheets/d/1vK6IgEqaqXniUJAQOOspiL_tx3EYTSXW1cUrMHAZFr8/edit?gid=0#gid=0
Перед сдачей проверьте код, напишите тесты. Не забудьте про PEP8, например, с помощью flake8. Задание нужно делать в jupyter notebook.

**Дедлайн - 14 ноября 10:00**

# Менеджер контекста для смены директории (cd)

Напишите класс менеджера контекста ChangeDir, который временно меняет текущую рабочую директорию на заданную. После выхода из контекста рабочая директория должна вернуться к предыдущей.

**Условия:**
1.	При входе в блок with менеджер контекста должен изменить текущую директорию на указанную.
2.	При выходе из блока with менеджер контекста должен вернуть рабочую директорию на исходное значение.
3.	Обработайте ситуацию, когда указанный путь не существует, с выводом сообщения об ошибке.

**Пример:**

```python
import os

print("Начальная директория:", os.getcwd())

try:
    with ChangeDir("/path/to/new/directory"):
        print("Внутри менеджера:", os.getcwd())
except FileNotFoundError as e:
    print(e)

print("После выхода:", os.getcwd())

Начальная директория: /Users/jdima/Apps/teaching/lectures/python/2024/itmo/7. manager context/homework
Ошибка: Директория '/path/to/new/directory' не существует.
Директория '/path/to/new/directory' не найдена.
После выхода: /Users/jdima/Apps/teaching/lectures/python/2024/itmo/7. manager context/homework
```

In [10]:
import os

class ChangeDir:
    def __init__(self, new_path):
        self.new_path = new_path
        self.original_path = None

    def __enter__(self):
        self.original_path = os.getcwd()
        if not os.path.exists(self.new_path):
            raise FileNotFoundError(f"Директория '{self.new_path}' не найдена.")
        os.chdir(self.new_path)

    def __exit__(self, exc_type, exc_value, traceback):
        os.chdir(self.original_path)

print("Начальная директория:", os.getcwd())

try:
    with ChangeDir("./sample_data"):
        print("Внутри менеджера:", os.getcwd())
except FileNotFoundError as e:
    print(e)

print("После выхода:", os.getcwd())

Начальная директория: /content
Внутри менеджера: /content/sample_data
После выхода: /content


In [19]:
import unittest

class TestChangeDir(unittest.TestCase):
    def test_change_dir(self):
        original_dir = os.getcwd()
        with ChangeDir(os.path.expanduser("~")):
            self.assertEqual(os.getcwd(), os.path.expanduser("~"))
        self.assertEqual(os.getcwd(), original_dir)

    def test_invalid_dir(self):
        with self.assertRaises(FileNotFoundError):
            with ChangeDir("/non/existent/directory"):
                pass

# Перенаправления вывода в файл

Напишите класс менеджера контекста RedirectOutput, который временно перенаправляет стандартный поток вывода stdout в указанный файл. После выхода из контекста вывод должен возвращаться в стандартный поток.

**Условия:**

1.	При входе в блок with менеджер контекста должен перенаправить вывод print в файл, указанный при создании объекта.
2.	При выходе из блока with вывод должен возвращаться в стандартный поток.
3.	Если файл уже существует, вывод должен дописываться к нему, а не перезаписывать его.

**Пример:**
```python
print("Это стандартный вывод")  # Должно выводиться в консоль

with RedirectOutput("output.txt"):
    print("Это вывод в файл")   # Должно записываться в файл "output.txt"

print("Снова стандартный вывод")  # Должно выводиться в консоль
```


In [2]:
#self.file = open(self.filename, 'a')
#sys.stdout = self.file  # Перенаправляем stdout в файл

In [3]:
import sys

class RedirectOutput:
    def __init__(self, filename):
        self.filename = filename
        self.original_stdout = None
        self.file = None

    def __enter__(self):
        self.original_stdout = sys.stdout
        self.file = open(self.filename, 'a', encoding='utf-8')
        sys.stdout = self.file
        return self.file

    def __exit__(self, exc_type, exc_value, traceback):
        sys.stdout = self.original_stdout
        if self.file:
            self.file.close()

# Пример использования
print("Это стандартный вывод")  # Должно выводиться в консоль

with RedirectOutput("output.txt"):
    print("Это вывод в файл")   # Должно записываться в файл "output.txt"

print("Снова стандартный вывод")  # Должно выводиться в консоль

Это стандартный вывод
Снова стандартный вывод


In [20]:
class TestRedirectOutput(unittest.TestCase):
    def test_redirect_output(self):
        test_file = "test_output.txt"
        with RedirectOutput(test_file):
            print("Это вывод в файл")
        with open(test_file, "r", encoding="utf-8") as f:
            content = f.read()
        self.assertIn("Это вывод в файл", content)
        os.remove(test_file)

# Замер времени выполнения кода

Напишите класс менеджера контекста Timer, который замеряет время выполнения кода внутри блока with. Менеджер должен выводить время выполнения в консоль по завершении блока. Для замера времени используйте модуль time.

**Условия:**
1. При входе в блок with менеджер контекста должен начинать отсчёт времени.
2. При выходе из блока with менеджер должен выводить в консоль время выполнения кода внутри блока в формате "Время выполнения: X.XXX секунд".
3. Опционально: добавить возможность передавать имя таймера при инициализации, чтобы можно было различать результаты замеров, если их несколько.

**Пример:**
```python
import time

with Timer("Задача 1"):
    time.sleep(1)  # Симуляция работы кода
[Задача 1] Время выполнения: 1.001 секунд
    
with Timer("Задача 2"):
    for i in range(1000000):
        pass
[Задача 2] Время выполнения: 0.034 секунд
```

In [4]:
import time

class Timer:
    def __init__(self, name=None):
        self.name = name
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        end_time = time.time()
        elapsed = end_time - self.start_time
        if self.name:
            print(f"[{self.name}] Время выполнения: {elapsed:.3f} секунд")
        else:
            print(f"Время выполнения: {elapsed:.3f} секунд")

# Пример использования
with Timer("Задача 1"):
    time.sleep(1)  # Симуляция работы кода

with Timer("Задача 2"):
    for i in range(1000000):
        pass

[Задача 1] Время выполнения: 1.001 секунд
[Задача 2] Время выполнения: 0.103 секунд


In [21]:
class TestTimer(unittest.TestCase):
    def test_timer(self):
        with Timer("Test Timer") as t:
            time.sleep(0.1)
        self.assertTrue(hasattr(t, "start_time"))

# Поглощение исключения

Напишите класс менеджера контекста SuppressExceptions, который подавляет указанные исключения внутри блока with, не прерывая выполнение программы. Если в блоке возникает исключение, которое не входит в список подавляемых, оно должно быть выброшено обычным образом.

**Условия:**
1.	При инициализации менеджера контекста нужно передавать типы исключений, которые будут подавляться.
2.	Если в блоке with возникает исключение из списка подавляемых, оно должно игнорироваться.
3.	Если возникает исключение, не входящее в список, оно должно быть выброшено.
4.	Опционально: после подавления исключения вывести сообщение о том, какое исключение было подавлено.


**Пример:**
```python
with SuppressExceptions(ZeroDivisionError, ValueError):
    print(1 / 0)  # Это исключение будет подавлено

with SuppressExceptions(TypeError):
    print(1 + "2")  # Это исключение будет подавлено

with SuppressExceptions(IndexError):
    print([1, 2, 3][5])  # Это исключение будет подавлено

print("Программа продолжает работать после блока with")
```

In [11]:
class SuppressExceptions:
    def __init__(self, *exception_types):
        self.exception_types = exception_types

    def __enter__(self):
        pass

    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is None:
            return False
        if issubclass(exc_type, self.exception_types):
            print(f"Исключение '{exc_type.__name__}' было подавлено")
            return True
        return False

# Пример использования
with SuppressExceptions(ZeroDivisionError, ValueError):
    print(1 / 0)  # Это исключение будет подавлено

with SuppressExceptions(TypeError):
    print(1 + "2")  # Это исключение будет подавлено

with SuppressExceptions(IndexError):
    print([1, 2, 3][5])  # Это исключение будет подавлено

print("Программа продолжает работать после блока with")

Исключение 'ZeroDivisionError' было подавлено
Исключение 'TypeError' было подавлено
Исключение 'IndexError' было подавлено
Программа продолжает работать после блока with


In [22]:
class TestSuppressExceptions(unittest.TestCase):
    def test_suppress_specific_exception(self):
        with SuppressExceptions(ZeroDivisionError):
            1 / 0

    def test_raise_unsuppressed_exception(self):
        with self.assertRaises(ValueError):
            with SuppressExceptions(ZeroDivisionError):
                raise ValueError("Это исключение должно быть выброшено")

# Создание временного файла
Напишите класс менеджера контекста TemporaryFile, который создаёт временный файл при входе в контекст и автоматически удаляет его при выходе. Менеджер должен позволять записывать и читать данные из файла в течение его существования в контексте.

**Условия:**
1.	При входе в блок with менеджер должен создавать временный файл и возвращать его объект для записи и чтения.
2.	При выходе из блока with временный файл должен автоматически удаляться.
3.	Имя файла должно быть уникальным и генерироваться автоматически.

**Пример**
```python
with TemporaryFile() as temp_file:
    temp_file.write(b"Временные данные\n")  # Записываем данные
    temp_file.seek(0)  # Возвращаемся в начало файла
    print(temp_file.read())  # Читаем данные из временного файла

print("Файл автоматически удалён")
```

In [18]:
import os
import tempfile
import random
import string

class TemporaryFile:
    def __init__(self):
        self.temp_file = None
        self.file_path = None

    def __enter__(self):
        random_name = "temp_" + ''.join(random.choices(string.ascii_letters + string.digits, k=12)) + ".tmp"
        self.file_path = os.path.join(tempfile.gettempdir(), random_name)
        self.temp_file = open(self.file_path, 'w+b')
        return self.temp_file

    def __exit__(self, exc_type, exc_value, traceback):
        if self.temp_file:
            self.temp_file.close()
            os.remove(self.file_path)
            print("Файл автоматически удалён")

# Пример использования
with TemporaryFile() as temp_file:
    temp_file.write(b"temporary data\n")  # Записываем данные
    temp_file.seek(0)  # Возвращаемся в начало файла
    print(temp_file.read().decode().rstrip("\n"))  # Читаем данные из временного файла

temporary data
Файл автоматически удалён


In [23]:
class TestTemporaryFile(unittest.TestCase):
    def test_file_creation_and_removal(self):
        with TemporaryFile() as temp_file:
            file_path = temp_file.name
            self.assertTrue(os.path.exists(file_path))
        self.assertFalse(os.path.exists(file_path))

    def test_write_and_read_file(self):
        with TemporaryFile() as temp_file:
            data = b"Hello, Temporary File!"
            temp_file.write(data)
            temp_file.seek(0)
            result = temp_file.read()
            self.assertEqual(result, data)

In [29]:
unittest.main(argv=[''], verbosity=2, exit=False)
None

test_change_dir (__main__.TestChangeDir)
Тест изменения текущей директории. ... ok
test_invalid_dir (__main__.TestChangeDir)
Тест обработки несуществующей директории. ... ok
test_redirect_output (__main__.TestRedirectOutput) ... ok
test_raise_unsuppressed_exception (__main__.TestSuppressExceptions) ... ok
test_suppress_specific_exception (__main__.TestSuppressExceptions) ... ok
test_file_creation_and_removal (__main__.TestTemporaryFile) ... ok
test_write_and_read_file (__main__.TestTemporaryFile) ... ok
test_timer (__main__.TestTimer) ... ok

----------------------------------------------------------------------
Ran 8 tests in 0.139s

OK


Исключение 'ZeroDivisionError' было подавлено
Файл автоматически удалён
Файл автоматически удалён
[Test Timer] Время выполнения: 0.100 секунд
